In [11]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from nltk.corpus import stopwords
from itertools import combinations
import string
from collections import Counter, defaultdict

In [12]:
path_connected_data = "C:\\Users\\tykun\\OneDrive\\Documents\\SchoolDocs\\VSCodeProjects\\connectedData\\"
path_temp_data = "C:\\Users\\tykun\\OneDrive\\Documents\\SchoolDocs\\VSCodeProjects\\connectedData\\temporaryData\\"
year = "1999"

full_path = f"{path_connected_data}{year}_gptDataframe.csv"
grouped_path = f"{path_temp_data}{year}_grouped.csv"
grouped_validated_path = f"{path_temp_data}{year}_grouped_validated.csv"
full_validation_path = f"{path_temp_data}full_validation.csv"

pd.options.display.max_rows = 400

In [13]:
#define keywords
keywords = {
    "Arts and Sciences": ["arts and science", "letters and science", "arts, sciences, and letters", "arts, sciences", "arts & sciences", "letters, sciences", "arts and letters", "arts & letters"],
    "Engineering": ["engineering", "mines", "aerospace", "aeronautics", "polytechnic studies", "aviation"],
    "Applied Science": ["applied science"],
    
    "Computer Science": ["computer science", "computer,", "software engineering", "and computer", "computing"],
    "Data Science": ["data science", "artificial intelligence", "machine learning"],
    "Technology": ["technology"],
    "Information": ["information science", "information management", "information systems", "information stud", "library science", "and information", "library stud", "informatics", "school of information", 
                    "information and", "information school"],
    "Archictecure": ["architecture", "architectural", "urban design", "urban planning", "environmental design", "environment and design", "design, construction"],
    

    "Behavioral Science": ["psychology", "psychological", "human science", "human ecology", "behavioral", "human and", "childgood stud"],
    "Social Sciences": ["social science", "social and", "social ecology"],
    "Natural Sciences": ['natural science', "mathematics", "chemistry", "biology", "mathematical", "physical science", "life science", "biological science", 
                        "biochemistry", "natural and", "basic", "college of science", "school of science", ", science, and", ", science and", "physics", "college of the sciences"],
    "Environmental and Earth Science": ["environmental science", "forestry", "natural resource", "college of forest", 
                              "school of forest", "school of environment ", "environmental and", "geology", "oceanography", "earth science", "geophysics", "seismology", "volcanology", "earth and" 
                              "paleontology", "mineral science", "geoscience", "earth science", "earth studies", "atmospheric", "college of the environment", "school of ecology", "school of environment"],
    "Marine Science": ["marine science", "marine stud", "ocean science", "oceanography", "oceanographic", "ocean and"],

    
    "Business": ["business", "commerce", "mba", "college of economics", "and economics", "finance", "entrepreneurship", "school of management", "and management", "accounting", "agribusiness", "economic,", "accountancy", "schools of management", "college of management"],
    "Hospitality": ["hospitality", "hotel administration", "hotel management", "restaurant management"],
    "Education": ["college of education", "school of education", "teaching", "teach", "curriculum development", "human development", "and education", ", education and"],
    "Criminal Justice": ["criminal justice", "criminology", "criminal", "forensics"],


    "Law": ["law", "legal studies"],
    "Policy and Affairs": ["public policy", "international policy", "public affairs", "international affairs", "environmental affairs", "urban affairs", "metropolitan affairs", "international service", "leadership studies", 
                           "diplomacy", "diplomat", "international studies", "public administration", "urban policy", "global studies", "international relations", "social policy", "peace studies", "global management", "policy studies",
                           "policy, planning", "school of leadership"],
    

    "Agriculture": ["agriculture", "agricultural", "agribusiness"],
    "Family and Consumer Science": ["family and", "home economics", "consumer science", "family science", "home"],
    "Culinary": ["culinary", "cooking", "gastronomy"],


    "Journalism": ["journalism"],
    "Communication": ["communication"],
    "Liberal Arts": ["liberal art", "liberal studies", "interdisciplinary studies", "general studies", "liberal and", "individualized study"],
    "Humanities": ["humanities", "history", "historical", "philosophy", "literature", "political"],
    "Fine Arts": ["fine arts", "music", "theater", "creative", "theatre", "film", "performing art", "the arts", "design art", "dramatic arts", "dance",
                   "visual arts", "school of drama", "recording arts", "ceramics", "fine and applied art", "school of design", "art and design", "fine", 
                   "cinema", "art ", "art, ", "studio art", "media art", "arts and performance", "drama", "arts and media", "textiles", "applied arts"],
    "Language and Culture": ["languages", "lingusitics", "cultures", "translational studies", "intercultural studies", "ethnic studies", "hebraic studies", "cultural stud", "asian studies", "culture ", " hawaiian knowledge", 
                             "jewish", "asian and pacific studies"],


    "Nursing": ["nursing"],
    "Physical Therapy and Nutrition": ["physical therapy", "nutrition", "physiotherapy", "nutritional therapy", "rehab", "human performance"],
    "Medical": ["medical", "pharmacy", "surgeons", "school of medicine", "college of medicine", "podiatric medicine", "osteopathic medicine", "human medicine"],
    "Pharmacy": ["pharmacy", "pharmacology"],
    "Dentistry" : ["dentistry", "dental", "denistry"],
    "Optometry" : ["optometry", "optics", "optical science"],
    "Veterinary" : ['veterinary'],
    "Audiology": ["audiology"],
    "Gerontology": ["gerontology"],
    "Pathology": ["pathology"],
    "Health Science": ["health science", "public health", "health profession", "college of health", "school of health", "biomedical science", "and health", "allied health", "physical education",
                        "health and", "health studies", "genome"],


    "Religious Studies": ["religious", "theology", "divinity", "biblical", "theological", "religion", "seminary", "religous", "christ college", "talmudic", "torah", "quran", "bible"],
    "Social Work": ["social work", "social welfare", "community service", "human service", "social service", "public service", "social administration"],
    

    "Graduate": ["graduate school", "graduate studies", "graduate college", "college of graduate", "school of graduate",  "advancing", "advanced study", "graduate division", "graduate and professional studies", "doctoral", "graduate study", "graduate "],
    "Continued Studies": ["extended studies", "extended education", "continuing education", "continued education", "continuing studies", "lifelong learning", "professional programs", "extended learning", "adult learning",
                           "adult education", "online learning", "adult student", "lifelong education", "continuing and", "university extension", "distance learning", "continuous education", "extended university"],
    "Professional Studies": ["professional studies", "professional development", "professional education", "school of professions"],
    "Administration": ["academic service", "academics", "academic program", "admissions", "admission", "enrollment service", "student life", "housing", "academic support", "advising", "enrollment management", 
                       "student relations", "academic computing", "academic records", "student service", "student affairs", "student development", "registrar", "financial aid", "media service", "library service", 
                       "university librar", "information service", "information access", "library access", "residence", "academic affairs", "library affairs", "academic succes", "first year student",
                       "undergraduate studies", "undergraduate education"],
    "Honor College": ["honor college", "honors college", "scholar program"],

    "Satellite Campus": ["campus"],
    "Foundation": ["foundation"],
    "Boards": ["board of", "regents", "trustees", "council of"],
    "Throwaway": [],
    "Hospital": ["hospital"],
    "University College": ["university college"],
    "Error": []
}

In [14]:
validated_df = pd.read_csv(grouped_validated_path)
validated_df.sort_values(by="Category")
validated_subinstitution_list = list(zip(validated_df["Institution"].tolist(), validated_df["SubInstitution"].tolist()))
validated_subinstitution_dict = {key: value for key, value in zip(validated_subinstitution_list, validated_df["Category"].tolist())}
print ((validated_subinstitution_dict))

{('Angelo State University', 'Student Life'): 'Administration', ('Aquinas College', 'Academics and Dean of the Faculty'): 'Administration', ('Aquinas College', 'Academic Programs and Services'): 'Administration', ('Arizona State University', 'University Libraries'): 'Administration', ('Arizona State University', 'ASU West Library'): 'Administration', ('Auburn University', 'University Library'): 'Administration', ('Austin Peay State University', 'Library and Media Services'): 'Administration', ('Baldwin Wallace College', 'Students'): 'Administration', ('Bard College', 'Admission'): 'Administration', ('Bard College', 'Students'): 'Administration', ('Barry University', 'Academic Computing'): 'Administration', ('Barry University', 'Student Services'): 'Administration', ('Barry University', 'Academic Records and University Registrar'): 'Administration', ('Barry University', 'Enrollment Services'): 'Administration', ('Barry University', 'Library Services and University Librarian'): 'Administ

In [15]:
grouped_df = pd.read_csv(grouped_path)
for index, row in grouped_df.iterrows():
    inst_sub = (row["Institution"], row["SubInstitution"])
    if row["Category"] == "Other" and inst_sub in validated_subinstitution_dict.keys():
        grouped_df.at[index, "Category"] = validated_subinstitution_dict[inst_sub]
#number of elements that have yet to be categorized
sorted_combined_df = grouped_df.sort_values(by="Category")
category_counter = Counter(sorted_combined_df['Category'])
num_other = category_counter['Other']
total = len(sorted_combined_df)
print("Remaining unclassified: ", 100*(num_other/total), "%")
sorted_combined_df.to_csv(grouped_path, index=False)

Remaining unclassified:  0.22038567493112945 %


In [16]:
validation_2009 = pd.read_csv(f"{path_temp_data}2009_grouped_validated.csv")
validation_2005 = pd.read_csv(f"{path_temp_data}2005_grouped_validated.csv")
validation_1999 = pd.read_csv(f"{path_temp_data}1999_grouped_validated.csv")


full_validation_list = pd.concat([validation_2005, validation_2009, validation_1999]).drop_duplicates()
full_validation_list = full_validation_list.drop_duplicates()
full_validation_list = full_validation_list.sort_values(by=["Category","Institution"])
full_validation_list.to_csv(full_validation_path, index = False)
